# Machine learning on wine

**Topics:** Text analysis, linear regression, logistic regression, text analysis, classification

**Datasets**

- **wine-reviews.csv** Wine reviews scraped from https://www.winemag.com/
- **Data dictionary:** just go [here](https://www.winemag.com/buying-guide/tenuta-dellornellaia-2007-masseto-merlot-toscana/) and look at the page

## The background

You work in the **worst newsroom in the world**, and you've had a hard few weeks at work - a couple stories killed, a few scoops stolen out from under you. It's not going well.

And because things just can't get any worse: your boss shows up, carrying a huge binder. She slams it down on your desk.

"You know some machine learning stuff, right?"

You say "no," but she isn't listening. She's giving you an assignment, the _worst assignment_...

> Machine learning is the new maps. Let's get some hits!
>
> **Do some machine learning on this stuff.**

"This stuff" is wine reviews.

## A tiny, meagre bit of help

You have a dataset. It has some stuff in it:

* **Numbers:**
    - Year published
    - Alcohol percentage
    - Price
    - Score
    - Bottle size
* **Categories:**
    - Red vs white
    - Different countries
    - Importer
    - Designation
    - Taster
    - Variety
    - Winery
* **Free text:**
    - Wine description

# Cleaning up your data

Many of these pieces - the alcohol, the year produced, the bottle size, the country the wine is from - aren't in a format you can use. Convert the ones to numbers that are numbers, and extract the others from the appropriate strings.

In [1]:
import pandas as pd


pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [4]:
from google.colab import files
uploaded = files.upload()

Saving wine-reviews.csv to wine-reviews.csv


In [5]:
df = pd.read_csv("wine-reviews.csv")
df.head(5)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating
0,https://www.winemag.com/buying-guide/artadi-20...,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,"$25, Buy Now",Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5%,750 ml,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review]
1,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2012 Stoller Vineyard Chardonnay (Du...,"A tiny production wine, this is rich, tart and...",Paul Gregutt,"$65, Buy Now",Stoller Vineyard,Chardonnay,"Dundee Hills, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
2,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2013 Ribbon Springs Vineyard Auxerro...,This is another fine vintage for this rare win...,Paul Gregutt,"$25, Buy Now",Ribbon Springs Vineyard,"Auxerrois, Other White","Ribbon Ridge, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
3,https://www.winemag.com/buying-guide/jcb-2011-...,90.0,JCB 2011 No. 11 Pinot Noir (Sonoma Coast),Light in color and lilting floral aromas of ro...,Virginie Boone,"$65, Buy Now",No. 11,Pinot Noir,"Sonoma Coast, Sonoma, California, US",JCB,13%,750 ml,Red,NaN,12/1/2014,Not rated yet [Add Your Review]
4,https://www.winemag.com/buying-guide/pazo-pond...,90.0,Pazo Pondal 2013 Albariño (Rías Baixas),"Alluring, inviting aromas of white flowers, me...",Michael Schachner,"$17, Buy Now",NaN,Albariño,"Rías Baixas, Galicia, Spain",Pazo Pondal,13%,750 ml,White,Vinaio Imports,12/1/2014,Not rated yet [Add Your Review]


In [6]:
df.shape

(42295, 16)

In [8]:
# cleaning price column
df.price = df.price.str.replace("$", "")

In [9]:
df.price = df.price.str.replace("Buy Now", "")

In [14]:
df.price = df.price.replace(',','', regex=True)

In [10]:
# cleaning alcohol column
df.alcohol = df.alcohol.replace('%','', regex=True)

In [11]:
# cleaning bottle size column
df['bottle size'] = df['bottle size'].str.replace('ml','')

In [13]:
# cleaning country column
df['country']= df.appellation.str.rsplit(',').str[-1]

In [15]:
# cleaning year column
df['year'] = df['date published'].str[-4:]

In [16]:
df['year'] = df['year'].astype(float)

In [17]:
df['year'] = 2021 - df['year']

In [ ]:
#df.isna().sum()

In [19]:
df.head(10)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,country,year
0,https://www.winemag.com/buying-guide/artadi-20...,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,25,Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5,750,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review],Spain,7.0
1,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2012 Stoller Vineyard Chardonnay (Du...,"A tiny production wine, this is rich, tart and...",Paul Gregutt,65,Stoller Vineyard,Chardonnay,"Dundee Hills, Willamette Valley, Oregon, US",Adelsheim,13.5,750,White,NaN,12/1/2014,Not rated yet [Add Your Review],US,7.0
2,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2013 Ribbon Springs Vineyard Auxerro...,This is another fine vintage for this rare win...,Paul Gregutt,25,Ribbon Springs Vineyard,"Auxerrois, Other White","Ribbon Ridge, Willamette Valley, Oregon, US",Adelsheim,13.5,750,White,NaN,12/1/2014,Not rated yet [Add Your Review],US,7.0
3,https://www.winemag.com/buying-guide/jcb-2011-...,90.0,JCB 2011 No. 11 Pinot Noir (Sonoma Coast),Light in color and lilting floral aromas of ro...,Virginie Boone,65,No. 11,Pinot Noir,"Sonoma Coast, Sonoma, California, US",JCB,13,750,Red,NaN,12/1/2014,Not rated yet [Add Your Review],US,7.0
4,https://www.winemag.com/buying-guide/pazo-pond...,90.0,Pazo Pondal 2013 Albariño (Rías Baixas),"Alluring, inviting aromas of white flowers, me...",Michael Schachner,17,NaN,Albariño,"Rías Baixas, Galicia, Spain",Pazo Pondal,13,750,White,Vinaio Imports,12/1/2014,Not rated yet [Add Your Review],Spain,7.0
5,https://www.winemag.com/buying-guide/mumm-napa...,90.0,Mumm Napa 2008 DVX Rosé Sparkling (Napa Valley),"Pretty peach in color, this 50-50 sparkling bl...",Virginie Boone,70,DVX Rosé,"Sparkling Blend, Sparkling","Napa Valley, Napa, California, US",Mumm Napa,12.5,750,Sparkling,NaN,12/1/2014,Not rated yet [Add Your Review],US,7.0
6,https://www.winemag.com/buying-guide/nuiton-be...,90.0,Nuiton-Beaunoy 2011 Clos du Chapitre Premier C...,The two-acre Clos du Chapitre vineyard is in t...,Roger Voss,N/A,Clos du Chapitre Premier Cru,Pinot Noir,"Gevrey-Chambertin, Burgundy, France",Nuiton-Beaunoy,13,750,Red,"Fruit of the Vines, Inc",12/1/2014,Not rated yet [Add Your Review],France,7.0
7,https://www.winemag.com/buying-guide/trapiche-...,90.0,Trapiche 2012 Broquel Cabernet Sauvignon (Mend...,"Spice, licorice and herbal notes complement re...",Michael Schachner,15,Broquel,Cabernet Sauvignon,"Mendoza, Mendoza Province, Argentina",Trapiche,14,750,Red,The Wine Group,12/1/2014,Not rated yet [Add Your Review],Argentina,7.0
8,https://www.winemag.com/buying-guide/zonin-201...,90.0,Zonin 2010 Amarone della Valpolicella,"Full-bodied and fresh, this offfers attractive...",Kerin O’Keefe,50,NaN,"Red Blends, Red Blends","Amarone della Valpolicella, Veneto, Italy",Zonin,15,750,Red,Zonin USA,12/1/2014,Not rated yet [Add Your Review],Italy,7.0
9,https://www.winemag.com/buying-guide/pali-2012...,90.0,Pali 2012 Cargasacchi Vineyard Pinot Noir (Sta...,"Round, savory aromas of orange-cranberry with ...",Matt Kettmann,56,Cargasacchi Vineyard,Pinot Noir,"Sta. Rita Hills, Central Coast, California, US",Pali,13.8,750,Red,NaN,12/1/2014,Not rated yet [Add Your Review],US,7.0


In [23]:

df.dropna(how='any', inplace=True)

In [24]:
df.shape

(13318, 18)

In [25]:
df.head(10)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,country,year
0,https://www.winemag.com/buying-guide/artadi-20...,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,25,Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5,750,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review],Spain,7.0
6,https://www.winemag.com/buying-guide/nuiton-be...,90.0,Nuiton-Beaunoy 2011 Clos du Chapitre Premier C...,The two-acre Clos du Chapitre vineyard is in t...,Roger Voss,N/A,Clos du Chapitre Premier Cru,Pinot Noir,"Gevrey-Chambertin, Burgundy, France",Nuiton-Beaunoy,13,750,Red,"Fruit of the Vines, Inc",12/1/2014,Not rated yet [Add Your Review],France,7.0
7,https://www.winemag.com/buying-guide/trapiche-...,90.0,Trapiche 2012 Broquel Cabernet Sauvignon (Mend...,"Spice, licorice and herbal notes complement re...",Michael Schachner,15,Broquel,Cabernet Sauvignon,"Mendoza, Mendoza Province, Argentina",Trapiche,14,750,Red,The Wine Group,12/1/2014,Not rated yet [Add Your Review],Argentina,7.0
10,https://www.winemag.com/buying-guide/peter-nic...,90.0,Peter Nicolay 2012 Erdener Treppchen Feinherb ...,"Honey-kissed peaches waft from this ripe, rich...",Anna Lee C. Iijima,20,Erdener Treppchen Feinherb,Riesling,"Mosel, Germany",Peter Nicolay,12,750,White,Saranty Imports,12/1/2014,Not rated yet [Add Your Review],Germany,7.0
11,https://www.winemag.com/buying-guide/pol-roger...,90.0,Pol Roger NV Réserve Brut (Champagne),"Full and ripe, this offers balance between ric...",Roger Voss,64,Réserve Brut,"Champagne Blend, Sparkling","Champagne, Champagne, France",Pol Roger,12.5,750,Sparkling,"Frederick Wildman & Sons, Ltd",12/1/2014,Not rated yet [Add Your Review],France,7.0
12,https://www.winemag.com/buying-guide/quinta-pa...,90.0,Quinta de Paços 2013 Casa do Capitão-mor Alvar...,Named after the 13th-century house at the cent...,Roger Voss,18,Casa do Capitão-mor,"Alvarinho, Albariño","Vinho Verde, Portugal",Quinta de Paços,13,750,White,Aidil Wines/Old World Import,12/1/2014,Not rated yet [Add Your Review],Portugal,7.0
13,https://www.winemag.com/buying-guide/quinta-no...,90.0,Quinta do Noval NV 10-Years-Old Tawny (Port),"This dry, balanced aged tawny is both fruity a...",Roger Voss,30,10-Years-Old Tawny,"Port, Port Blend","Port, Portugal",Quinta do Noval,20,750,Port/Sherry,Vintus LLC,12/1/2014,Not rated yet [Add Your Review],Portugal,7.0
15,https://www.winemag.com/buying-guide/maso-mart...,90.0,Maso Martis 2007 Brut Riserva Sparkling (Trento),"A blend of 70% Pinot Nero and 30% Chardonnay, ...",Kerin O’Keefe,52,Brut Riserva,"Sparkling Blend, Sparkling","Trento, Northeastern Italy, Italy",Maso Martis,12.5,750,Sparkling,Solstars,12/1/2014,Not rated yet [Add Your Review],Italy,7.0
16,https://www.winemag.com/buying-guide/maso-mart...,90.0,Maso Martis 2009 Dosaggiozero Riserva Sparklin...,"Made with no dosage, this is a blend of 70% Ch...",Kerin O’Keefe,52,Dosaggiozero Riserva,"Sparkling Blend, Sparkling","Trento, Northeastern Italy, Italy",Maso Martis,12.5,750,Sparkling,Solstars,12/1/2014,Not rated yet [Add Your Review],Italy,7.0
17,https://www.winemag.com/buying-guide/maximin-g...,90.0,Maximin Grünhäuser 2012 Herrenberg Kabinett Ri...,"Dried oregano and sage notes lend a dusty, sav...",Anna Lee C. Iijima,34,Herrenberg Kabinett,Riesling,"Mosel, Germany",Maximin Grünhäuser,8,750,White,Loosen Bros. USA,12/1/2014,Not rated yet [Add Your Review],Germany,7.0


In [26]:
# df.dtypes

## What might be interesting in this dataset?

Maybe start out playing around _without_ machine learning. Here are some thoughts to get you started:

* I've heard that since the 90's wine has gone through [Parkerization](https://www.estatewinebrokers.com/blog/the-parkerization-of-wine-in-the-1990s-and-beyond/), an increase in production of high-alcohol, fruity red wines thanks to the influence of wine critic Robert Parker.
* Red and white wines taste different, obviously, but people always use [goofy words to describe them](https://winefolly.com/tutorial/40-wine-descriptions/)
* Once upon a time in 1976 [California wines proved themselves against France](https://en.wikipedia.org/wiki/Judgment_of_Paris_(wine)) and France got very angry about it

In [27]:
# whether a wine is white or red based on the wine's description

In [28]:
#whether alcohol content depends on country of origin

In [29]:
# finding out correlation between wine_points and price

In [30]:
# finding out correlation between wine_points and wine desc

In [31]:
# finding out correlation between wine_points and variety

In [32]:
# finding out correlation between wine desc and category

In [33]:
# finding out correlation between variety and country

## But machine learning?

Well, you can usually break machine learning down into a few different things. These aren't necessarily perfect ways of categorizing things, but eh, close enough.

* **Predicting a number**
    - Linear regression
    - For example, how does a change in unemployment translate into a change in life expectancy?
* **Predicting a category** (aka classification)
    - Lots of algos options: logistic regression, random forest, etc
    - For example, predicting cuisines based on ingredients
* **Seeing what influences a numeric outcome**
    - Linear regression since the output is a number
    - For example, minority and poverty status on test scores 
* **Seeing what influences a categorical outcome**
    - Logistic regression since the output is a category
    - Race and car speed for if you get a waring vs ticket
    - Wet/dry pavement and car weight if you survive or not in a car crash)

We have numbers, we have categories, we have all sorts of stuff. **What are some ways we can mash them together and use machine learning?**

### Brainstorm some ideas

Use the categories above to try to come up with some ideas. Be sure to scroll up where I break down categories vs numbers vs text!

**I'll give you one idea for free:** if you don't have any ideas, start off by creating a classifier that determines whether a wine is white or red based on the wine's description.

In [34]:
# Linear regression - how does the price affect wine ratings?

In [35]:
# Predicting a category - predicting wine category based on wine description 

In [36]:
# Seeing what influences a numeric outcome - 
# country and variety on ratings

You can also go to https://library.columbia.edu and see if you can find some academic papers about wine. I'm sure they'll inspire you! (and they might even have some ML ideas in them you can steal, too)

# Implement 2 of your machine learning ideas

## IDEA 1: Linear regression - how does the year affect wine ratings?

In [ ]:
df_LR = df[['year', 'wine_points']]

In [ ]:
df_LR.head()

,year,wine_points
0,7.0,90.0
6,7.0,90.0
7,7.0,90.0
10,7.0,90.0
11,7.0,90.0


In [ ]:
df_LR.isna().sum()

year           0
wine_points    0
dtype: int64

In [ ]:
df_LR.dtypes

year           float64
wine_points    float64
dtype: object

In [ ]:
df_LR.year = df_LR.year.astype(float)

C:\Users\kanin\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
import statsmodels.api as sm

# What effect does the year have on ratings?
X = df_LR[['year']]
y = df_LR.wine_points

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

In [ ]:
results.params

const    90.725928
year     -0.210720
dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            wine_points   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     975.3
Date:                Thu, 01 Apr 2021   Prob (F-statistic):          1.08e-206
Time:                        21:53:45   Log-Likelihood:                -34334.
No. Observations:               13318   AIC:                         6.867e+04
Df Residuals:                   13316   BIC:                         6.869e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         90.7259      0.064   1412.621      0.000      90.600      90.852
year          -0.2107      0.007    -31.230      0.000      -0.224      -0.197
==============================================================================
Omnibus:                      220.820   Durbin-Watson:                   0.381
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              204.689
Skew:                          -0.262   Prob(JB):                     3.57e-45
Kurtosis:                       2.693   Cond. No.                         22.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# ANALYSIS
# For every "year" a wine ages, it gets 0.15 less "points"
# I'm guessing that's b/c older wine tends to be more expensive


In [ ]:
import statsmodels.formula.api as smf
import numpy as np

model = smf.ols(formula='wine_points ~ np.multiply(year, 100)', data=df_LR)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            wine_points   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     975.3
Date:                Thu, 01 Apr 2021   Prob (F-statistic):          1.08e-206
Time:                        21:53:45   Log-Likelihood:                -34334.
No. Observations:               13318   AIC:                         6.867e+04
Df Residuals:                   13316   BIC:                         6.869e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 90.7259      0.064   1412.621      0.000      90.600      90.852
np.multiply(year, 100)    -0.0021   6.75e-05    -31.230      0.000      -0.002      -0.002
==============================================================================
Omnibus:                      220.820   Durbin-Watson:                   0.381
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              204.689
Skew:                          -0.262   Prob(JB):                     3.57e-45
Kurtosis:                       2.693   Cond. No.                     2.21e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.21e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## IDEA 2: Predicting a category - predicting wine category based on wine description 

In [37]:
df_P = df[['wine_desc', 'category']]

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

# Make a vectorizer
vectorizer = CountVectorizer()

# Learn and count the words in df.content
matrix = vectorizer.fit_transform(df_P.wine_desc)

# Convert the matrix of counts to a dataframe
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())

In [39]:
words_df.head(1)

,000,01,02,02s,04,05,06,06s,07,08,09,10,100,1000,11,110,114,117,12,125,1290,12th,13,13th,14,14th,15,150,1500,150th,155,15th,16,1610,1667,166th,1674,16th,17,1756,1763,1789,1791,17th,18,180,1800,1806,1850s,1855,1894,18th,19,190,1900,1901,1904,1910,1914,1920s,1922,1930,1930s,1932,1934,1935,1936,1939,1940s,1946,1947,1950,1951,1954,1955,1971,1974,1980s,1983,1984,1985,1986,1990s,1991,1994,1995,1996,1997,1998,1999,19th,20,200,2000,2000s,2001,2001s,2002,2002s,2003,...,wrinkles,write,writers,written,wrong,wrought,würzgarten,xarel,xarello,xavier,ximenez,ximénez,xinomavro,xiv,yalumba,yangarra,yarra,yarrow,yauquén,yealands,year,yearns,years,yeast,yeastiness,yeasts,yeasty,yecla,yellow,yellowing,yellowish,yering,yes,yesterday,yet,yianni,yield,yielded,yielding,yields,yogev,yogurt,yonne,york,you,young,younger,youngest,youngster,your,yours,yourself,youth,youthful,youthfully,ysios,yummy,yung,yup,yuzu,yves,zamora,zantho,zap,zaps,zealand,zellenberg,zeller,zelma,zero,zest,zestier,zestiness,zesty,zibibbo,zierfandler,zimmermann,zin,zinfandel,zing,zinger,zings,zingy,zip,zippiness,zippy,zips,zone,zuccardi,zull,zuri,zweigelt,zwerithaler,zwiegelt,zédé,zéro,élevage,émilion,épernay,ürziger
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
df_P['is_red'] = (df_P['category'] == 'Red').astype(int)
df_P.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,wine_desc,category,is_red
0,"Inky, minerally aromas of blackberry, black pl...",Red,1
6,The two-acre Clos du Chapitre vineyard is in t...,Red,1
7,"Spice, licorice and herbal notes complement re...",Red,1
10,"Honey-kissed peaches waft from this ripe, rich...",White,0
11,"Full and ripe, this offers balance between ric...",Sparkling,0


In [41]:
X = words_df
y = df_P.is_red

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [43]:
from sklearn.tree import DecisionTreeClassifier 
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [44]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not red', 'red'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not red,Predicted red
Is not red,1609,82
Is red,266,1373


In [45]:
from sklearn.svm import LinearSVC 
clf = LinearSVC(max_iter=10000)
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [51]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not red', 'red'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not red,Predicted red
Is not red,1609,82
Is red,83,1556


In [52]:
!pip install eli5

In [53]:
import eli5

eli5.show_weights(clf, feature_names=vectorizer.get_feature_names())

In [54]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_P['category_label'] = le.fit_transform(df_P.category)
df_P.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,wine_desc,category,is_red,category_label
0,"Inky, minerally aromas of blackberry, black pl...",Red,1,3
6,The two-acre Clos du Chapitre vineyard is in t...,Red,1,3
7,"Spice, licorice and herbal notes complement re...",Red,1,3
10,"Honey-kissed peaches waft from this ripe, rich...",White,0,6
11,"Full and ripe, this offers balance between ric...",Sparkling,0,5


In [55]:
X = words_df
y = df_P.category_label

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [57]:
from sklearn.svm import LinearSVC 
clf = LinearSVC(max_iter=10000)
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [58]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(le.classes_)
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted Dessert,Predicted Fortified,Predicted Port/Sherry,Predicted Red,Predicted Rose,Predicted Sparkling,Predicted White
Is Dessert,32,0,2,4,1,2,28
Is Fortified,1,1,3,0,0,0,1
Is Port/Sherry,3,0,38,13,0,1,4
Is Red,1,0,8,1599,11,12,24
Is Rose,0,0,0,25,72,17,15
Is Sparkling,1,0,0,15,13,264,60
Is White,13,0,3,28,5,39,971


In [59]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(le.classes_)
scores = pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)
scores.style.background_gradient(cmap='YlGnBu')

,Predicted Dessert,Predicted Fortified,Predicted Port/Sherry,Predicted Red,Predicted Rose,Predicted Sparkling,Predicted White
Is Dessert,0.463768,0.000000,0.028986,0.057971,0.014493,0.028986,0.405797
Is Fortified,0.166667,0.166667,0.500000,0.000000,0.000000,0.000000,0.166667
Is Port/Sherry,0.050847,0.000000,0.644068,0.220339,0.000000,0.016949,0.067797
Is Red,0.000604,0.000000,0.004834,0.966163,0.006647,0.007251,0.014502
Is Rose,0.000000,0.000000,0.000000,0.193798,0.558140,0.131783,0.116279
Is Sparkling,0.002833,0.000000,0.000000,0.042493,0.036827,0.747875,0.169972
Is White,0.012276,0.000000,0.002833,0.026440,0.004721,0.036827,0.916903
